$k$-NN
===

**Autor:** Mateus de Jesus Mendes

# Introdução

Este *Jupyter Notebook* tem por objetivo implementar o algoritmo $k$-NN para induzir um modelo a partir dos elementos do *dataset* `xtb_dataset.csv`.

## Fundamentação Teórica

### Contextualização geral


O algoritmo *$k$-Nearest Neighbors* ($k$-NN) é um estimador não paramétrico, baseado em instâncias e pertencente à classe dos métodos de aprendizado supervisionado por similaridade **\[[1, 2]()\]**. Diferentemente de modelos paramétricos tradicionais, o $k$-NN não realiza uma etapa explícita de treinamento no sentido clássico: não há ajuste de coeficientes, nem inferência de uma função global que relacione entradas e saídas. Em vez disso, o modelo armazena o conjunto de dados de treinamento e realiza predições exclusivamente com base na proximidade geométrica entre amostras no espaço de atributos **\[[3]()\]**.

Essa característica faz do $k$-NN um método conceitualmente simples, porém poderoso, especialmente adequado para problemas nos quais a relação entre variáveis é altamente local, potencialmente não linear e difícil de ser capturada por funções globais suaves **\[[1, 4]()\]**. Por outro lado, essa mesma propriedade impõe limitações claras em termos de escalabilidade, interpretabilidade e extrapolação fora do domínio amostrado.

### Representação do espaço de atributos

Considere um conjunto de dados de treinamento:
$$
\mathcal{D} = {(\mathbf{x}*i, y_i)}*{i=1}^{N}
$$

Em que:
- $\mathbf{x}_i \in \mathbb{R}^d$: Vetor de *features* da (i)-ésima amostra
- $y_i \in \mathbb{R}$: Valor alvo associado
- $N$: Número total de observações 

O algoritmo kNN interpreta cada vetor $\mathbf{x}_i$ como um ponto em um espaço métrico de dimensão (d). A noção de similaridade entre duas amostras é formalizada por meio de uma função de distância $d(\mathbf{x}, \mathbf{x}')$, cuja escolha é um hiperparâmetro central do método **\[[2, 3]()\]**.


### Métricas de distância


A distância entre duas observações $\mathbf{x}$ e $\mathbf{x}'$ pode ser definida de diversas formas. Entre as métricas mais utilizadas destacam-se **\[[1, 4]()\]**:

- **Distância Euclidiana**:
$$
d_2(\mathbf{x}, \mathbf{x}') = \sqrt{\sum_{j=1}^{d} (x_j - x'_j)^2}
$$

- **Distância Manhattan**:
$$
d_1(\mathbf{x}, \mathbf{x}') = \sum_{j=1}^{d} |x_j - x'_j|
$$

A escolha da métrica influencia diretamente a geometria do espaço de vizinhança. Métricas do tipo $\ell_1$ (Manhattan) tendem a ser mais robustas em espaços de alta dimensionalidade e em cenários nos quais as *features* possuem naturezas físicas distintas ou distribuições heterogêneas **\[[4, 5]()\]**.

### Definição dos $k$ vizinhos mais próximos

Dada uma nova observação $\mathbf{x}_*$, o algoritmo identifica o subconjunto:
$$
\mathcal{N}_k(\mathbf{x}_*) \subset \mathcal{D}
$$

O subconjunto encontrado é composto pelos $k$ pontos do conjunto de treinamento que minimizam a distância $d(\mathbf{x}_*, \mathbf{x}_i)$ **\[[1]()\]**.

O hiperparâmetro $k$ controla diretamente o viés–variância do estimador:
- Valores pequenos de $k$ produzem modelos altamente flexíveis, com baixo viés (*bias*) e alta variância;
- Valores maiores de $k$ suavizam as predições, aumentando o viés e reduzindo a sensibilidade a ruído local **\[[2, 6]()\]**.

### Regra de predição para regressão

No contexto de regressão, a predição associada a $\mathbf{x}_*$ é obtida como uma média dos valores alvo dos vizinhos selecionados. Na forma mais simples (*uniform weights*), tem-se:
$$
\hat{y}(\mathbf{x}_*) = \frac{1}{k} \sum_{(\mathbf{x}_i, y_i) \in \mathcal{N}_k(\mathbf{x}_*)} y_i
$$

Uma generalização amplamente empregada utiliza ponderação por distância, na qual vizinhos mais próximos exercem maior influência na predição **\[[3, 5]()\]**:

$$
\hat{y}(\mathbf{x}_*) = \frac{\sum_{i \in \mathcal{N}_k(\mathbf{x}_*)}w_i(\mathbf{x}_*)y_i}{\sum_{i \in \mathcal{N}_k(\mathbf{x}_*)}w_i(\mathbf{x}_*)}
$$

Com pesos tipicamente definidos por:
$$
w_i(\mathbf{x}_*) = \frac{1}{d(\mathbf{x}_*, \mathbf{x}_i) + \varepsilon}
$$

em que $\varepsilon > 0$ evita singularidades numéricas **\[[1]()\]**.

### Natureza não paramétrica e ausência de treinamento explícito

O $k$-NN é classificado como um método *lazy learning*, pois não constrói um modelo explícito durante o treinamento **\[[2]()\]**. Todo o custo computacional é transferido para a etapa de inferência, na qual é necessário calcular distâncias entre a amostra de teste e todas — ou muitas — das amostras de treinamento.

Essa característica implica:
- Custo de memória proporcional a $\mathcal{O}(Nd)$,
- Custo de inferência proporcional a $\mathcal{O}(Nd)$ por predição, na forma ingênua **\[[4]()\]**.

Estruturas de dados como *KD-trees* e *Ball trees* podem reduzir esse custo em situações específicas, embora sua eficiência decaia em espaços de alta dimensionalidade **\[[5]()\]**.

### Sensibilidade à escala e pré-processamento

Como o critério central do $k$-NN é puramente geométrico, o algoritmo é altamente sensível à escala das *features*. Variáveis com maior amplitude numérica tendem a dominar o cálculo de distâncias, independentemente de sua relevância semântica ou física **\[[1]()\]**.

Por esse motivo, é prática comum empregar normalização ou padronização dos dados. Contudo, em cenários específicos — como quando as escalas originais carregam significado físico coerente — o uso do espaço original pode ser justificável, desde que empiricamente validado **\[[4]()\]**.

### Interpretabilidade e limitações conceituais

Diferentemente de modelos lineares ou baseados em árvores, o $k$-NN não fornece parâmetros interpretáveis, coeficientes ou decomposições explícitas de importância das *features*. A predição emerge exclusivamente da distribuição local dos dados, o que inviabiliza análises diretas de interpretabilidade baseadas em coeficientes ou métodos como SHAP **\[[6, 7]()\]**.

Assim, o $k$-NN deve ser compreendido como um estimador empírico local, cuja força reside na fidelidade interpolativa e cuja principal limitação é a ausência de uma representação explícita da relação funcional subjacente entre variáveis.

### Considerações finais

O algoritmo $k$-NN constitui um método conceitualmente simples, matematicamente direto e empiricamente poderoso para problemas de regressão e classificação baseados em similaridade. Ao abdicar de hipóteses paramétricas globais, o $k$-NN é capaz de capturar padrões locais complexos e não lineares, desde que o espaço de atributos seja adequadamente amostrado **\[[1, 2]()\]**.

Entretanto, essa flexibilidade vem acompanhada de limitações claras em termos de interpretabilidade, escalabilidade e extrapolação. Dessa forma, o $k$-NN é particularmente valioso como *baseline* empírico de alta fidelidade e como ferramenta complementar a modelos paramétricos e interpretáveis, contribuindo para uma avaliação abrangente do espaço de soluções possíveis em problemas de aprendizado supervisionado.

# Metodologia

### Importações

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import root_mean_squared_error
from sklearn.feature_selection import VarianceThreshold
from sklearn.exceptions import ConvergenceWarning

import warnings

from optuna import create_study
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner
from optuna.trial import FixedTrial
from optuna.exceptions import TrialPruned
from optuna import load_study

import os

### Definições Globais

Definições de parâmetros globais usados ao longo desse *Jupyter Notebook*, a fim de assegurar clareza metodológica e reprodutibilidade:

In [30]:
RANDOM_SEED = 88
PATH = '../../dataset_processing/xtb_dataset.csv'
TRAIN_SIZE = 0.8
STUDY_NAME = 'knn'

### Leitura dos Dados

Leitura do *dataset*:

In [31]:
df = pd.read_csv(PATH)

X = df.drop(columns=['Delta'])
y = df['Delta']

### Pré-processamento dos Dados

Divisão do *dataset* em subconjuntos de dados de treino e teste, considerando as *features* e o *target*:

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=TRAIN_SIZE, random_state=RANDOM_SEED)

### Treino & Otimização de Hiperparâmetros

Instância do modelo com todos os hiperparâmetros e tratamentos a serem usados pelo `optuna`.

In [33]:
def inst_knn(trial, n_features):


    # Pré-processamento disponível

    pre_processing = trial.suggest_categorical(
        "pre_processing", [None, "VT", "PCA"]
    )

    steps = []


    # Normalização (sempre obrigatória)

    steps.append(("scale", StandardScaler()))


    # Variance Threshold
    if pre_processing == "VT":
        threshold = trial.suggest_float(
            "variance_threshold", 0.0, 1e-3
        )
        steps.append((
            "vt", VarianceThreshold(threshold=threshold)
        ))


    # PCA
    elif pre_processing == "PCA":
        max_comp = min(n_features, 8)
        n_comp = trial.suggest_int(
            "pca_components", 2, max_comp
        )
        steps.append((
            "pca", PCA(n_components=n_comp)
        ))


    # Hiperparâmetros do kNN
    n_neighbors = trial.suggest_int(
        "n_neighbors", 3, 50
    )

    weights = trial.suggest_categorical(
        "weights", ["uniform", "distance"]
    )

    metric = trial.suggest_categorical(
        "metric", ["euclidean", "manhattan", "minkowski"]
    )

    p = 2
    if metric == "minkowski":
        p = trial.suggest_int("p", 1, 2)


    # Modelo kNN
    steps.append((
        "knn", KNeighborsRegressor(
            n_neighbors=n_neighbors,
            weights=weights,
            metric=metric,
            p=p,
            n_jobs=1
        )
    ))

    model = make_pipeline(*[s[1] for s in steps])

    return model


Função objetivo para validação cruzada:

In [34]:
def objective_function(trial, X, y, NUM_FOLDS=5):
    n_features = X.shape[1]
    cv = KFold(
        n_splits=NUM_FOLDS,
        shuffle=True,
        random_state=RANDOM_SEED
    )

    rmse_folds = []

    for i, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        model = inst_knn(trial, n_features=n_features)

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            rmse = root_mean_squared_error(y_test, y_pred)

        rmse_folds.append(rmse)

        trial.report(np.mean(rmse_folds), step=i + 1)
        if trial.should_prune():
            raise TrialPruned()

    return float(np.mean(rmse_folds))

Funções para validação cruzada aninhada:

In [35]:
def nested_cv_fold(fold_idx, X, y, outer_splits, inner_splits, n_trials, pasta_estudos):
    outer_cv = KFold(n_splits=outer_splits, shuffle=True, random_state=RANDOM_SEED)

    for fold, (idx_train, idx_test) in enumerate(outer_cv.split(X, y)):
        if fold != fold_idx:
            continue

        print(f"\nFold externo {fold + 1}/{outer_splits}")

        X_train, X_test = X.iloc[idx_train], X.iloc[idx_test]
        y_train, y_test = y.iloc[idx_train], y.iloc[idx_test]

        db_path = os.path.join(
            pasta_estudos,
            f"{STUDY_NAME}_fold_{fold + 1}.db"
        )

        def inner_objective(trial):
            return objective_function(
                trial,
                X_train,
                y_train,
                NUM_FOLDS=inner_splits
            )

        study = create_study(
            study_name=f"{STUDY_NAME}_fold_{fold + 1}",
            direction="minimize",
            sampler=TPESampler(seed=RANDOM_SEED),
            pruner=HyperbandPruner(
                min_resource=1,
                max_resource=inner_splits,
                reduction_factor=2
            ),
            storage=f"sqlite:///{db_path}",
            load_if_exists=True
        )

        study.optimize(
            inner_objective,
            n_trials=n_trials,
            n_jobs=1,
            show_progress_bar=False
        )

        print("Melhores hiperparâmetros encontrados:")
        print(study.best_params)

        best_model = inst_knn(
            FixedTrial(study.best_params),
            n_features=X_train.shape[1]
        )

        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning)
            best_model.fit(X_train, y_train)
            y_pred = best_model.predict(X_test)
            rmse_test = root_mean_squared_error(y_test, y_pred)

        print(f"RMSE de teste externo (fold {fold + 1}): {rmse_test:.4f}")
        return rmse_test

In [36]:
def nested_cv(X, y, outer_splits=5, inner_splits=3, n_trials=200, studies_folder=f"optuna_{STUDY_NAME}_studies"):
    os.makedirs(studies_folder, exist_ok=True)
    outer_scores = []

    for fold in range(outer_splits):
        rmse_fold = nested_cv_fold(
            fold_idx=fold,
            X=X,
            y=y,
            outer_splits=outer_splits,
            inner_splits=inner_splits,
            n_trials=n_trials,
            studies_folder=studies_folder
        )
        outer_scores.append(rmse_fold)

    outer_scores = np.array(outer_scores)

    print("\n========================")
    print("Resultados do Nested CV:")
    print(f"RMSE médio: {outer_scores.mean():.4f}")
    print(f"Desvio padrão: {outer_scores.std():.4f}")
    print("========================")

    return outer_scores

Execução dos estudos de otimização de hiperparâmetros do `optuna`:

> **📌 Observação:** O seguinte código está comentado como célula Markdown por apresentar elevado custo computacional. Desse modo, é recomendada execução do *script* `kNN.py` utilizando computação de alto desempenho (HPC) a fim de garantir eficiência e estabilidade computacional na execução dos estudos de otimização. Após isso, basta o diretório `svr_knn_studies` para o mesmo diretório desse *Jupyter Notebook* e executar as células seguintes.

---
```python
results = nested_cv(X_train, y_train, outer_splits=5, inner_splits=3, n_trials=100)
```
---

Acesso aos estudos de otimização, para extração dos melhores valores dos hiperparâmetros e o desempenho obtido:

In [37]:
studies_folder = f"optuna_{STUDY_NAME}_studies"
db_files = [f for f in os.listdir(studies_folder) if f.endswith(".db")]

print("Arquivos .db encontrados:")
for f in db_files:
    print("-", f)

Arquivos .db encontrados:
- knn_fold_1.db
- knn_fold_2.db
- knn_fold_3.db
- knn_fold_4.db
- knn_fold_5.db


Melhores hiperparâmetros obtidos durante a otimização:

In [38]:
results = []

for file in db_files:
    path = os.path.join(studies_folder, file)
    study_name = file.replace('.db', '')
    
    try:
        study = load_study(study_name=study_name, storage=f"sqlite:///{path}")
        best_trial = study.best_trial
        
        print(f'======= {study_name} =======')
        print(f"  Melhor valor: {best_trial.value:.7f}")
        print(f"  Parâmetros: {best_trial.params}\n")
        
        results.append({
            "fold": study_name,
            "best_value": best_trial.value,
            "params": best_trial.params
        })
    except Exception as e:
        print(f"Não foi possível carregar {study_name}: {e}")

# Exibe melhores resultados
if results:
    df_results = pd.DataFrame(results)
    print(f"RMSE médio: {np.mean(df_results['best_value']):.7f}")
    print(f"Desvio padrão: {np.std(df_results['best_value']):.7f}")
else:
    print("Nenhum estudo pôde ser carregado.")

======= knn_fold_1 =======
  Melhor valor: 0.0588802
  Parâmetros: {'pre_processing': None, 'n_neighbors': 4, 'weights': 'distance', 'metric': 'manhattan'}

======= knn_fold_2 =======
  Melhor valor: 0.0580648
  Parâmetros: {'pre_processing': None, 'n_neighbors': 4, 'weights': 'distance', 'metric': 'manhattan'}

======= knn_fold_3 =======
  Melhor valor: 0.0583172
  Parâmetros: {'pre_processing': None, 'n_neighbors': 4, 'weights': 'distance', 'metric': 'manhattan'}

======= knn_fold_4 =======
  Melhor valor: 0.0605619
  Parâmetros: {'pre_processing': None, 'n_neighbors': 4, 'weights': 'distance', 'metric': 'manhattan'}

======= knn_fold_5 =======
  Melhor valor: 0.0581896
  Parâmetros: {'pre_processing': None, 'n_neighbors': 4, 'weights': 'distance', 'metric': 'manhattan'}

RMSE médio: 0.0588027
Desvio padrão: 0.0009227


Desempenho do melhor modelo obtido:

In [39]:
# df já contém os melhores trials de cada fold
best_fold = df_results.loc[df_results['best_value'].idxmin()]  # menor RMSE
print("Melhor fold:", best_fold['fold'])
print("Melhor desempenho (RMSE):", best_fold['best_value'])
print("Melhores parâmetros:", best_fold['params'])

Melhor fold: knn_fold_2
Melhor desempenho (RMSE): 0.05806481966872507
Melhores parâmetros: {'pre_processing': None, 'n_neighbors': 4, 'weights': 'distance', 'metric': 'manhattan'}


### Teste

Treino e teste finais para avaliação do melhor modelo obtido:

In [40]:
knn_model = KNeighborsRegressor(n_neighbors=4, weights='distance', metric='manhattan')

best_model = Pipeline([
    ("scaler", StandardScaler()),
    ("estimator", knn_model)
])
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

rmse = root_mean_squared_error(y_test, y_pred)
print(f'RMSE do melhor modelo: {rmse}')

RMSE do melhor modelo: 0.03311216742845702


# Resultados

O melhor modelo induzido a partir do algoritmo $k$-NN durante a otimização de hiperparâmetros do `optuna` obteve $\mathrm{RMSE}$ de $\approx 0.06$, dotado dos seguintes hiperparâmetros tabulados e com o conjunto de dados original, sem qualquer tipo de pré-processamento das *features*.

| Hiperparâmetros | Valor       |
|-----------------|-------------|
| `n_neighbors`   | $4$         |
| `weights`       | `distance`  |
| `metric`        | `manhattan` |

Por fim, ao desenvolver o teste final, o modelo induzido com os hiperparâmetros supracitados obteve $\mathrm{RMSE}$ de $\approx 0.03$, objetivamente menor, mas com mesma ordem de grandeza.

# Conclusão

A análise do desempenho do modelo $k$-NN, fundamentada exclusivamente em métricas quantitativas e no comportamento algorítmico intrínseco do método, conduz a uma conclusão complementar — e conceitualmente distinta — daquela obtida para os modelos paramétricos e interpretáveis empregados ao longo do projeto.

O melhor modelo induzido via otimização de hiperparâmetros com o `optuna` apresentou desempenho preditivo elevado, com $\mathrm{RMSE} \approx 0.06$ durante a etapa de seleção e $\mathrm{RMSE} \approx 0.03$ no teste final. Embora o valor obtido no conjunto final seja objetivamente menor, ambos pertencem à mesma ordem de grandeza, o que indica ausência de instabilidade severa entre fases de validação e teste, bem como uma generalização adequada dentro do regime de vizinhança local característico do $k$-NN.

A configuração ótima — $k = 4$, ponderação por distância e métrica Manhattan — revela aspectos estruturais importantes do espaço de atributos. O baixo número de vizinhos indica que o modelo se beneficia de relações altamente locais, explorando regiões densas e coerentes do espaço de *features*, em vez de tendências globais suaves. A escolha de `weights='distance'` reforça essa interpretação, ao atribuir maior influência a observações mais próximas, reduzindo o impacto de pontos potencialmente ruidosos ou marginalmente similares. Já a métrica Manhattan sugere que diferenças absolutas coordenada a coordenada são mais informativas do que distâncias euclidianas globais, o que é consistente com um espaço de atributos heterogêneo, possivelmente composto por variáveis de diferentes naturezas físicas e escalas.

É particularmente relevante notar que o melhor desempenho foi obtido sem qualquer tipo de pré-processamento das *features*. Esse resultado indica que o próprio arranjo geométrico dos dados no espaço original já é suficientemente informativo para permitir uma boa discriminação local entre instâncias. Ao mesmo tempo, essa característica evidencia a sensibilidade do $k$-NN à escala e à distribuição das variáveis, reforçando que o bom desempenho observado depende fortemente da estrutura específica do dataset, e não de uma robustez algorítmica generalizável no sentido estrito.

Diferentemente dos modelos lineares regularizados e do SVR, o $k$-NN não admite uma decomposição explícita de contribuições individuais das *features*, inviabilizando análises do tipo SHAP. Consequentemente, sua elevada performance não decorre da identificação de uma subestrutura física interpretável de baixa dimensionalidade, mas sim da capacidade de interpolação local em regiões bem amostradas do espaço de dados. Nesse sentido, o modelo atua mais como um estimador não paramétrico de alta fidelidade local do que como um mecanismo de inferência estrutural.

Em síntese, o modelo $k$-NN demonstra excelente capacidade preditiva para o dataset em questão, sendo particularmente eficaz na captura de padrões locais finos sem impor hipóteses funcionais explícitas. Todavia, esse desempenho vem acompanhado de limitações claras em termos de interpretabilidade física, extrapolação e robustez fora do domínio amostrado. Assim, o $k$-NN se consolida como um modelo de referência empírica de alto desempenho, útil para estabelecer limites superiores práticos de erro, mas conceitualmente complementar — e não substituto — aos modelos paramétricos e interpretáveis que fundamentam a compreensão física do problema ao longo do projeto.

# Referências

[1] HASTIE, Trevor; TIBSHIRANI, Robert; FRIEDMAN, Jerome. *The Elements of Statistical Learning: Data Mining, Inference, and Prediction*. 2. ed. New York: Springer, 2009.

[2] BISHOP, Christopher M. *Pattern Recognition and Machine Learning*. New York: Springer, 2006.

[3] DUDOIT, Sandrine; FRIDLYAND, Jane. Classification in microarray experiments. *Statistical Analysis of Gene Expression Microarray Data*. Boca Raton: Chapman & Hall/CRC, 2003.

[4] JAMES, Gareth et al. *An Introduction to Statistical Learning*. 2. ed. New York: Springer, 2021.

[5] PEDREGOSA, Fabian et al. Scikit-learn: Machine Learning in Python. *Journal of Machine Learning Research*, v. 12, p. 2825–2830, 2011.

[6] COVER, Thomas; HART, Peter. Nearest neighbor pattern classification. *IEEE Transactions on Information Theory*, v. 13, n. 1, p. 21–27, 1967.

[7] MOLNAR, Christoph. *Interpretable Machine Learning*. 2. ed. 2022. Disponível em: [https://christophm.github.io/interpretable-ml-book/](https://christophm.github.io/interpretable-ml-book/). Acesso em: 26 Dez. 2025.